In [1]:
import wave
import keras
import pyaudio
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot

from librosa import piptrack

import IPython.display as ipd

In [45]:
CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2 
RATE = 44100 #sample rate
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "C:\\Users\\Dhairya Hindoriya\\OneDrive\\Desktop\\SER\\demo_audio\\sample.wav"
emotions=["Anger","disgust","fear","happy","Neutral", "sad", "surprise"]

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [46]:
ipd.Audio('C:\\Users\\Dhairya Hindoriya\\OneDrive\\Desktop\\SER\\demo_audio\\sample.wav')



Load The SER Model

In [47]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('C:\\Users\\Dhairya Hindoriya\\OneDrive\\Desktop\\SER\\utils\\model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Trained_Models/Speech_Emotion_Recognition_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [48]:
demo_audio_path = 'C:\\Users\\Dhairya Hindoriya\\OneDrive\\Desktop\\SER\\demo_audio\\sample.wav'

In [49]:



from feature_extraction import get_audio_features
demo_mfcc, demo_pitch, demo_mag, demo_chrom = get_audio_features(demo_audio_path,20000)

In [52]:
mfcc = pd.Series(demo_mfcc)
pit = pd.Series(demo_pitch)
mag = pd.Series(demo_mag)
C = pd.Series(demo_chrom)
demo_audio_features = pd.concat([mfcc,pit,mag,C],ignore_index=True)

In [53]:
demo_audio_features= np.expand_dims(demo_audio_features, axis=0)
demo_audio_features= np.expand_dims(demo_audio_features, axis=2)

In [54]:
demo_audio_features.shape

(1, 65, 1)

In [55]:
livepreds = loaded_model.predict(demo_audio_features, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 0s 151ms/step


In [56]:
livepreds

array([[1.000000e+00, 0.000000e+00, 0.000000e+00, 4.226766e-37,
        0.000000e+00, 0.000000e+00, 0.000000e+00]], dtype=float32)

In [57]:
#emotions=["anger","disgust","fear","happy","neutral", "sad", "surprise"]
index = livepreds.argmax(axis=1).item()
index

0

In [58]:
emotions[index]

'Anger'